# Swarm simulator

## Introduction

This notebook introduces a new swarm simulator. A key objective is to develop an efficient simulator using the Numpy library. The ideas behind the simulator are introduced gradually and illustrated by plotting the results using Matplotlib. Some boiler-plate code is introduced to assist with the computations and plotting.

In [ ]:
"""
Some boiler-plate to assist with plotting and animation
"""
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import plot_helper as ph
import numpy as np

A swarm comprises a set of *agents*. Each agent is defined by a number of *attributes*, including position, cohesion field, repulsion field, etc. Agent attributes will be introduced as we go along.

The principal attribute of an agent is its position in 2-D Euclidean space. We can specify an agent's position as a point, using the usual Cartesian coordinates or, equivalently, as a vector whose tail is located at the origin. For convenience and efficiency of use with Numpy, we model an agent's position as a column vector. For example, a swarm comprising a single agent, `b0`,
having an x-coordinate of 3 and a y-coordinate of 2 can be modelled as a column vector of shape (2,1), in which row 0 gives the x-coordinate, and row 1 the y-coordinate, of the single agent `b0` in column 0.

In [ ]:
"""
Show an agent at position (3,2) as a vector and as a point.

Note ph.plot_vector expects a list of points as its first argument
and draws vectors to all of them
"""
b = np.array([
    [3],             # x-coordinate
    [2]              # y-coordinate
]) 
ph.plot_vector(b.T)
plt.plot(b[0, 0], b[1, 0], 'ro', markersize=2)
plt.text(3, 2.1, 'b0')

Notice that `plot_vector()` requires the *transpose* of `b`, computed as `b.T`. The transpose operator, `b.T`, is implemented very efficiently in Numpy. No array data are copied. A new instance of the metadata is created in which the strides are adjusted to achieve the transposition. The transpose operator is both a convenient and an efficient mechanism for accessing all attributes of an agent in our representation.

In [ ]:
print(f"b is \n{b}")
print(f"The shape of b is {b.shape}")
print(f"The transpose of b is {b.T}")
print(f"The shape of b.T is {b.T.shape}")

A second agent, $b1$, at position $(1.5, -1)$ can be introduced.

In [ ]:
b = np.array([
    [3, 1.5],            # x-coordinates
    [2, -1]              # y-coordinates
])
ph.plot_vector(b.T)
plt.plot(b[0,0], b[1,0], 'ro', markersize=2)
plt.plot(b[0,1], b[1,1], 'go', markersize=2)
plt.text(3, 2.1, 'b0')
plt.text(1.5, -1.3, 'b1')

The vector from $b_0$ to $b_1$, denoted $\overrightarrow{b_0 b_1}$, is the vector, $x$, such that $b_0 + x = b_1$, i.e. $x = b_1 - b_0$. 

In [ ]:
x = b[:,1] - b[:,0]                                                        
vectors = np.append(b.T, [x], axis=0)                                     
tails = np.zeros_like(vectors)                                              # initially all vector tails are at the origin
tails[2] = vectors[0]                                                       # set the tail of the x vector to b_0
ph.plot_vector(vectors, tails, color=[ph.darkblue, ph.darkblue, ph.pink])   # plot the vectors
plt.plot(b[0,0], b[1,0], 'ro', markersize=2)                                # plot b_0 as a red circle
plt.plot(b[0,1], b[1,1], 'go', markersize=2)                                # plot b_1 as a green circle
plt.text(3, 2.1, 'b0')                                                      # write some helpful text labels
plt.text(2, -1.3, 'b1')

Similarly, the vector from $b_1$ to $b_0$, denoted $\overrightarrow{b_1 b_0}$, is given by $b_0 - b_1$.

In [ ]:
x = b[:,0] - b[:,1]
vectors = np.append(b.T, [x], axis=0)
tails = np.zeros_like(vectors)
tails[2] = b[:,1]
ph.plot_vector([b[:,0], b[:,1], x.T], tails, color=[ph.darkblue, ph.darkblue, ph.pink])
plt.plot(b[0,0], b[1,0], 'ro', markersize=2)
plt.plot(b[0,1], b[1,1], 'go', markersize=2)
plt.text(3, 2.1, 'b0')
plt.text(2, -1.3, 'b1')

The *magnitude* of the vector, $\overrightarrow{b_0 b_1}$, can be obtained by considering the line from $b_0$ to $b_1$ as the hypotenuse of a right-angled triangle whose other sides are parallel to the axes of the coordinate system. The `numpy` function `hypot`  returns the length of the hypotenuse, given the lengths of the other 2 sides, e.g. let $b_0 = (4, 2)$ and $b_1 = (1, -2)$, then the vector $\overrightarrow{b_0 b_1}$ has the magnitude shown below.

In [ ]:
b = np.array([
    [4, 1],            # x-coordinates
    [2, -2]            # y-coordinates
])
x = b[:,1] - b[:,0]
vectors = np.append(b.T, [x], axis=0)
tails = np.zeros_like(vectors)
tails[2] = vectors[0]
ph.plot_vector(vectors, tails, color=[ph.darkblue, ph.darkblue, ph.pink])
plt.plot(b[0,0], b[1,0], 'ro', markersize=2)
plt.plot(b[0,1], b[1,1], 'go', markersize=2)
plt.text(4, 2.1, 'b0')
plt.text(1, -2.3, 'b1')
plt.plot([1,4,4], [-2,-2,2], 'k--', linewidth=0.5)
plt.text(2.5, -1.8, '3')
plt.text(2.5, -1.8, '3')
plt.text(3.8, 0.5, '4')
plt.text(2.5, -0.5, '5')
magx = np.hypot(b[0,0] - b[0,1], b[1,0] - b[1,1])
print(f"The magnitude of the vector from b0 to b1 is {magx}")

We denote the magnitude of $\overrightarrow{b_1 b_2}$ by $\lVert \overrightarrow{b_1 b_2} \rVert$.

## Cohesion and Repulsion Fields

Agents in a swarm have two main goals: 

    1. to stay close to other agents, and 
    2. to avoid bumping into other agents.
    
The first goal involves defining a 'cohesion' field, $C_b$, for each agent $b$. The cohesion field of $b$ is specified as a circle of given radius, centred at $b$. Any agent $b'$ that is positioned within the cohesion field of $b$ inclines $b$ to move towards $b'$. The second goal involves defining a 'repulsion' field, $R_b$, in a similar manner to the definition of the cohesion field. Any agent $b'$ that is positioned within the repulsion field of $b$ inclines $b$ to move away from $b'$. Notice that an agent $b'$ that is positioned both within  $b$'s cohesion field *and* its repulsion field will cause $b$ to have conflicting inclinations: both to move towards and to move away from $b'$. The final movement of $b$ depends on the 'strength' of these inclinations.

Two new rows are added to the swarm representation. One row gives the cohesion field for each agent. The other defines the repulsion field. 

Note: A swarm can be represented in the simulator in a variety of ways. The goal here is to find a representation that leads to an efficient implementation of the simulator using Numpy. Here we choose a representation based on a 2-D array in which each row models a single attribute for all agents and each column models all attributes for a single agent, e.g.

| | b0 | b1 | ... | bn |
|---|---|---|---|---|
|x  |   |   |   |   |
|y  |   |   |   |   |
|C  |   |   |   |   |
|R  |   |   |   |   |
|. |   |   |   |   |
|.  |   |   |   |   |
|.  |   |   |   |   |

This is not the most convenient representation when plotting a few agents from a small swarm but it is hoped that it allows efficient implementation of the major operations in the simulator, by taking advantage of Numpy's vectorised operators.

We define a 'helper' function, `plot_field()`, to assist in the illustration of cohesion and repulsion fields.

In [ ]:
def plot_field(radius=1.0, fmt='b-', *, linewidth=0.5, pos=(0.0, 0.0)):
    '''
    Draw a circle of specified radius on the current axis
    
    :param radius: the radius of the circle
    :param fmt: the colour and style of line for drawing the circle
    :param linewidth: the width of the line for drawing the circle
    :param pos: the location of the centre of the circle
    '''
    theta = np.linspace(0, 2*np.pi, 100)
    x1 = radius*np.cos(theta) + pos[0]
    x2 = radius*np.sin(theta) + pos[1]
    ax = plt.gca()
    ax.plot(x1, x2, fmt, linewidth=linewidth)
    ax.set_aspect(1)


Now consider an agent, $b_0$, at the origin, with a repulsion field of radius 5 and a cohesion field of radius 7. This can be illustrated as follows:

In [ ]:
b = np.array([
    [0], # x coordinate
    [0], # y coordinate
    [7], # cohesion field radius
    [5], # repulsion field radius
])
ph.plot_vector([], limit=11.0)               # just draw the grid - no vectors
plot_field(b[2], 'b--')                      # draw the cohesion field of b[0]
plot_field(b[3], 'r--')                      # draw the repulsion field of b[0]
plt.plot(b[0], b[1], 'ko', markersize=2)     # draw the point for b[0]

## Cohesion vectors

Let's introduce a 2nd agent, $b_1$, into the previous scenario. We'll place $b_1$ at $(3, 3)$, which is within the cohesion field of $b_0$, and observe its effect on $b_0$. 

At this point, notice that it is important to be able to determine, for any agent, which agents are within its cohesion and repulsion fields. The necessary computation is introduced here. We begin by computing the pairwise distances between agents and determining, for each agent pair, whether one agent is within the cohesion / repulsion fields of the other. 

Notice that some constants are introduced to identify the rows of agent attributes in the swarm array. This aids the readability and maintainability of the code, without adversely affecting its performance too much. Some rows are introduced into the array to hold the results of computations of cohesion and repulsion vectors. It is expected that this approach will simplify the saving of swarm states when that functionality is introduced later.

In [ ]:
# Define some useful array row constants for agent attributes
POS_X  = 0    # x-coordinates of agents' position 
POS_Y  = 1    # y-coordinates of agents' position
COH_X  = 2    # x-coordinates of cohesion vectors
COH_Y  = 3    # y-coordinates of cohesion vectors
REP_X  = 4    # x-coordinates of repulsion vectors
REP_Y  = 5    # y-coordinates of repulsion vectors
CF     = 6    # cohesion field radii
RF     = 7    # repulsion field radii

b = np.array([
    [0.0, 4.0], # x-coordinates of agents' position
    [0.0, 2.0], # y-coordinates of agents' position
    [0.0, 0.0], # x-coordinates of cohesion vector
    [0.0, 0.0], # y-coordinates of cohesion vector
    [0.0, 0.0], # x-coordinates of repulsion vector
    [0.0, 0.0], # y-coordinates of repulsion vector
    [7.0, 7.0], # cohesion field radii
    [5.0, 5.0], # repulsion field radii
])
xv = np.subtract.outer(b[POS_X], b[POS_X])  # compute all pairs x-differences
yv = np.subtract.outer(b[POS_Y], b[POS_Y])  # compute all pairs y-differences

# compute all pairwise vector magnitudes
mag = np.hypot(xv, yv)              # all pairs magnitudes

# compute the cohesion neighbours
coh_n = mag <= b[CF]               # test for those pairs of agents for which one is within the cohesion field of the other  
np.fill_diagonal(coh_n, False)     # no agent is a cohesion neighbour of itself
nr_coh_n = np.sum(coh_n, axis = 0) # number of cohesion neighbours

# compute the x-differences and y-differences for cohesion vectors
xv_coh = np.where(coh_n, xv, 0.0)   # where coh_n is True take the corresponding value from xv, otherwise 0.0
yv_coh = np.where(coh_n, yv, 0.0)   # where coh_n is True take the corresponding value from yv, otherwise 0.0

# compute the cohesion vectors 
b[COH_X] = xv_coh.sum(axis=0)                       # sum the x-differences 
b[COH_Y] = yv_coh.sum(axis=0)                       # sum the y-differences
b[COH_X:COH_Y+1] /= np.maximum(nr_coh_n, 1)         # divide by the number of cohesion neighbours

# do plotting of results
ph.plot_vector([], limit=11.0)                          # just draw the grid - no vectors
plot_field(b[CF, 0], 'b--')                             # draw the cohesion field of b[0]
plot_field(b[RF, 0], 'r--')                             # draw the repulsion field of b[0]
plt.plot(b[POS_X], b[POS_Y], 'ko', markersize=2)        # draw the agent points 
coh_v_0 = np.column_stack((xv_coh.T[0], yv_coh.T[0]))   # get cohesion vector for agent 0 - just for plotting            
ph.plot_vector(coh_v_0, color=ph.green, newfig=False)   # draw the cohesion vector from b[0] to b[1]

It's worth having a look at some of the intermediate data structures in the previous example.

The `xv` (resp. `yv`) array holds all pairwise x-differences (resp. y-differences). The cell `xv[i,j]` is set to `b[POS_X, i] - b[POS_X, j]`, i.e. it gives the difference in x-value between the positions of agent `i` and agent `j`; Similarly, `yv[i,j]` is set to `b[POS_Y, i] - b[POS_Y, j]`, i.e. it gives the difference in y-value between the positions of agent `i` and agent `j`. Note that the x- and y-values for the vector $\overrightarrow{b_i b_j}$ are to be found at `xv[j,i]` and `yv[j,i]`, respectively. The `mag` array holds all pairwise magnitudes. The distance between agent `i` and agent `j`, or equivalently, the magnitude of the vector 
$\overrightarrow{b_i b_j}$ is to be found at `mag[j,i]`. Of course, in any sane world, $\lVert\overrightarrow{b_i b_j}\rVert = \lVert\overrightarrow{b_j b_i}\rVert$ but you can't be too careful!

In [ ]:
print(f"xv is \n {xv}")
print(f"yv is \n {yv}")
print(f"mag is \n {mag}")

The array `coh_n` is a boolean array where `coh_n[i, j]` is `True` if the position of agent i lies within the cohesion field of agent j, i.e. agent i is a cohesion *neighbour* of agent j. So `coh_n[:, j]` is `True` for all cohesion neighbours of agent `j`. The array `nr_coh_n` gives the number of cohesion neighbours of each agent. Note `np.sum()` counts every `True` as 1 and every `False` as 0. We are summing along `axis=0`, i.e. down the rows, so, for example, `nr_coh_n[j]` gives the number of cohesion neighbours of agent `j`.

In [ ]:
print(f"coh_n is \n {coh_n}")
print(f"nr_coh_n is \n {nr_coh_n}")

The array `xv_coh` (resp. `yv_coh`) is the same as `xv` (resp. `yv`) in those cells, `xv[i, j]` (resp. `yv[i, j]`), where `coh_n[i, j]` is `True` and is 0.0 in the remaining cells. The final x- and y-values of the cohesion vectors are computed by summing down the rows (axis=0) in `xv_coh` and `yv_coh` and dividing by the number of cohesion neighbours. See below.

In [ ]:
print(f"xv_coh is \n {xv_coh}")
print(f"yv_coh is \n {yv_coh}")
print(f"b[COH_X:COH_Y+1] is \n {b[COH_X:COH_Y+1]}")

##  Repulsion vectors

The presence of $b_1$ within the cohesion field of $b_0$ gives $b_0$ an inclination to move towards $b_1$. 
The 'strength' of the inclination is given by $\lVert\overrightarrow{b_0 b_1}\rVert$. Notice also that $b_1$ lies within the repulsion field of $b_0$. This gives $b_0$ an inclination to move away from $b_1$. This is calculated as 
$(\frac{\lVert\overrightarrow{b_0 b_1}\rVert}{R_{b_0}} - 1)\overrightarrow{b_0 b_1}$. The factor $(\frac{\lVert\overrightarrow{b_0 b_1}\rVert}{R_{b_0}} - 1)$ is used to reverse the direction, and to dampen the effect, of $\overrightarrow{b_0 b_1}$, giving $b_0$ a 'gentle' inclination to move away from $b_1$. This can be illustrated by adding a new vector to the previous figure, as follows:

In [ ]:
# compute the repulsion neighbours
rep_n = mag <= b[RF]
np.fill_diagonal(rep_n, False)     # no agent is a repulsion neighbour of itself
nr_rep_n = np.sum(rep_n, axis = 0) # number of repulsion neighbours

# compute the x-differences and y-differences  for repulsion vectors
rscalar = mag / b[RF] - 1
xv_rep = np.where(rep_n, xv * rscalar, 0.0)
yv_rep = np.where(rep_n, yv * rscalar, 0.0)

# compute the resultant repulsion vectors 
b[REP_X] = xv_rep.sum(axis=0)                       # sum the x-differences 
b[REP_Y] = yv_rep.sum(axis=0)                       # sum the y-differences
b[REP_X:REP_Y+1] /= np.maximum(nr_rep_n, 1)         # divide by the number of repulsion neighbours

# plot the results
ph.plot_vector([], limit=11.0)                                        # just draw the grid - no vectors
plot_field(b[CF, 0], 'b--')                                           # draw the cohesion field of b[0]
plot_field(b[RF, 0], 'r--')                                           # draw the repulsion field of b[0]
plt.plot(b[POS_X], b[POS_Y], 'ko', markersize=2)                      # draw the agent points 
coh_v_0 = np.column_stack((xv_coh.T[0], yv_coh.T[0]))                 # get cohesion vectors for agent 0 - just for plotting            
ph.plot_vector(coh_v_0, color=ph.green, newfig=False)                 # draw the cohesion vector from b[0] to b[1]
rep_v_0 = np.column_stack((xv_rep.T[0], yv_rep.T[0]))                 # get repulsion vectors for agent 0 - just for plotting            
ph.plot_vector(rep_v_0, color=ph.red, newfig=False)                   # draw the repulsion vector from b[0] to b[1]


At this stage, the repulsion effect is negligible (you can just about see a tiny red arrow pointing away from the origin). However, observe what happens when $b_1$ approaches closer to $b_0$.

In [ ]:
# Define some useful array row constants for agent attributes
POS_X  = 0    # x-coordinates of agents' position 
POS_Y  = 1    # y-coordinates of agents' position
COH_X  = 2    # x-coordinates of cohesion vectors
COH_Y  = 3    # y-coordinates of cohesion vectors
REP_X  = 4    # x-coordinates of repulsion vectors
REP_Y  = 5    # y-coordinates of repulsion vectors
CF     = 6    # cohesion field radii
RF     = 7    # repulsion field radii

b = np.array([
    [0.0, 1.0], # x-coordinates of agents' position
    [0.0, 0.5], # y-coordinates of agents' position
    [0.0, 0.0], # x-coordinates of cohesion vector
    [0.0, 0.0], # y-coordinates of cohesion vector
    [0.0, 0.0], # x-coordinates of repulsion vector
    [0.0, 0.0], # y-coordinates of repulsion vector
    [7.0, 7.0], # cohesion field radii
    [5.0, 5.0], # repulsion field radii
])
xv = np.subtract.outer(b[POS_X], b[POS_X])  # compute all pairs x-differences
yv = np.subtract.outer(b[POS_Y], b[POS_Y])  # compute all pairs y-differences

# compute all pairwise vector magnitudes
mag = np.hypot(xv, yv)              # all pairs magnitudes

# compute the cohesion neighbours
coh_n = mag <= b[CF]               # test for those pairs of agents for which one is within the cohesion field of the other  
np.fill_diagonal(coh_n, False)     # no agent is a cohesion neighbour of itself
nr_coh_n = np.sum(coh_n, axis = 0) # number of cohesion neighbours

# compute the x-differences and y-differences for cohesion vectors
xv_coh = np.where(coh_n, xv, 0.0)
yv_coh = np.where(coh_n, yv, 0.0)

# compute the cohesion vectors 
b[COH_X] = xv_coh.sum(axis=0)                       # sum the x-differences 
b[COH_Y] = yv_coh.sum(axis=0)                       # sum the y-differences
b[COH_X:COH_Y+1] /= np.maximum(nr_coh_n, 1)         # divide by the number of cohesion neighbours

# compute the repulsion neighbours
rep_n = mag <= b[RF]
np.fill_diagonal(rep_n, False)     # no agent is a repulsion neighbour of itself
nr_rep_n = np.sum(rep_n, axis = 0) # number of repulsion neighbours

# compute the x-differences  and y-differences for repulsion vectors
rscalar = mag / b[RF] - 1
xv_rep = np.where(rep_n, xv * rscalar, 0.0)
yv_rep = np.where(rep_n, yv * rscalar, 0.0)

# compute the resultant repulsion vectors 
b[REP_X] = xv_rep.sum(axis=0)                       # sum the x-differences 
b[REP_Y] = yv_rep.sum(axis=0)                       # sum the y-differences
b[REP_X:REP_Y+1] /= np.maximum(nr_rep_n, 1)         # divide by the number of repulsion neighbours

# plot the results
ph.plot_vector([], limit=11.0)                                        # just draw the grid - no vectors
plot_field(b[CF, 0], 'b--')                                           # draw the cohesion field of b[0]
plot_field(b[RF, 0], 'r--')                                           # draw the repulsion field of b[0]
plt.plot(b[POS_X], b[POS_Y], 'ko', markersize=2)                      # draw the agent points 
coh_v_0 = np.column_stack((xv_coh.T[0], yv_coh.T[0]))                 # get cohesion vectors for agent 0 - just for plotting            
ph.plot_vector(coh_v_0, color=ph.green, newfig=False)                 # draw the cohesion vector from b[0] to b[1]
rep_v_0 = np.column_stack((xv_rep.T[0], yv_rep.T[0]))                 # get repulsion vectors for agent 0 - just for plotting            
ph.plot_vector(rep_v_0, color=ph.red, newfig=False)                   # draw the repulsion vector from b[0] to b[1]

Now the repulsion effect is a little stronger.
Puzzlingly, however, as $b_1$ continues to approach $b_0$, drawing closer to it, even to the point of collision, the repulsion effect eventually begins to grow *weaker*. We'll return to this anomaly later.

Now consider what happens if a third agent is added to this scenario, at position $(-2, 3)$ with cohesion field $7$ and repulsion field $5$.

In [ ]:
# Define some useful array row constants for agent attributes
POS_X  = 0    # x-coordinates of agents' position 
POS_Y  = 1    # y-coordinates of agents' position
COH_X  = 2    # x-coordinates of cohesion vectors
COH_Y  = 3    # y-coordinates of cohesion vectors
REP_X  = 4    # x-coordinates of repulsion vectors
REP_Y  = 5    # y-coordinates of repulsion vectors
CF     = 6    # cohesion field radii
RF     = 7    # repulsion field radii

b = np.array([
    [0.0, 1.0, -2.0], # x-coordinates of agents' position
    [0.0, 0.5, 3.0],  # y-coordinates of agents' position
    [0.0, 0.0, 0.0],  # x-coordinates of cohesion vector
    [0.0, 0.0, 0.0],  # y-coordinates of cohesion vector
    [0.0, 0.0, 0.0],  # x-coordinates of repulsion vector
    [0.0, 0.0, 0.0],  # y-coordinates of repulsion vector
    [7.0, 7.0, 7.0],  # cohesion field radii
    [5.0, 5.0, 5.0],  # repulsion field radii
])
xv = np.subtract.outer(b[POS_X], b[POS_X])  # compute all pairs x-differences
yv = np.subtract.outer(b[POS_Y], b[POS_Y])  # compute all pairs y-differences

# compute all pairwise vector magnitudes
mag = np.hypot(xv, yv)              # all pairs magnitudes

# compute the cohesion neighbours
coh_n = mag <= b[CF]               # test for those pairs of agents for which one is within the cohesion field of the other  
np.fill_diagonal(coh_n, False)     # no agent is a cohesion neighbour of itself
nr_coh_n = np.sum(coh_n, axis = 0) # number of cohesion neighbours

# compute the x-differences and y-differences for cohesion vectors
xv_coh = np.where(coh_n, xv, 0.0)
yv_coh = np.where(coh_n, yv, 0.0)

# compute the resultant cohesion vectors 
b[COH_X] = xv_coh.sum(axis=0)                       # sum the x-differences 
b[COH_Y] = yv_coh.sum(axis=0)                       # sum the y-differences
b[COH_X:COH_Y+1] /= np.maximum(nr_coh_n, 1)         # divide by the number of cohesion neighbours

# compute the repulsion neighbours
rep_n = mag <= b[RF]
np.fill_diagonal(rep_n, False)     # no agent is a repulsion neighbour of itself
nr_rep_n = np.sum(rep_n, axis = 0) # number of repulsion neighbours

# compute the x-differences and y-differences for repulsion vectors
rscalar = mag / b[RF] - 1
xv_rep = np.where(rep_n, xv * rscalar, 0.0)
yv_rep = np.where(rep_n, yv * rscalar, 0.0)

# compute the resultant repulsion vectors 
b[REP_X] = xv_rep.sum(axis=0)                       # sum the x-differences 
b[REP_Y] = yv_rep.sum(axis=0)                       # sum the y-differences
b[REP_X:REP_Y+1] /= np.maximum(nr_rep_n, 1)         # divide by the number of repulsion neighbours

# plot the results
ph.plot_vector([], limit=11.0)                          # just draw the grid - no vectors
plot_field(b[CF, 0], 'b--')                             # draw the cohesion field of b[0]
plot_field(b[RF, 0], 'r--')                             # draw the repulsion field of b[0]
plt.plot(b[POS_X], b[POS_Y], 'ko', markersize=2)        # draw the agent points 
coh_v_0 = np.column_stack((xv_coh.T[0], yv_coh.T[0]))   # get cohesion vectors for agent 0 - just for plotting            
ph.plot_vector(coh_v_0, color=ph.green, newfig=False)   # draw the cohesion vectors from b[0] to b[1] and b[2]
rep_v_0 = np.column_stack((xv_rep.T[0], yv_rep.T[0]))   # get repulsion vectors for agent 0 - just for plotting 
ph.plot_vector(rep_v_0, color=ph.red, newfig=False)     # draw the repulsion vectors from b[0] to b[1] and b[2]

Now $b_0$ experiences cohesion and repulsion effects from *both* agents within its cohesion and repulsion fields (again, we note that it is puzzling that the 'strength' of repulsion caused by the closer agent is less that that caused by the agent that is further away). The net cohesion (resp. repulsion) effect on $b_0$ is given by the mean of the sum of the cohesion (resp. repulsion) effects arising from $b_1$ and $b_2$, as follows:

In [ ]:
# Define some useful array row constants for agent attributes
POS_X  = 0    # x-coordinates of agents' position 
POS_Y  = 1    # y-coordinates of agents' position
COH_X  = 2    # x-coordinates of cohesion vectors
COH_Y  = 3    # y-coordinates of cohesion vectors
REP_X  = 4    # x-coordinates of repulsion vectors
REP_Y  = 5    # y-coordinates of repulsion vectors
CF     = 6    # cohesion field radii
RF     = 7    # repulsion field radii

b = np.array([
    [0.0, 1.0, -2.0], # x-coordinates of agents' position
    [0.0, 0.5, 3.0],  # y-coordinates of agents' position
    [0.0, 0.0, 0.0],  # x-coordinates of cohesion vector
    [0.0, 0.0, 0.0],  # y-coordinates of cohesion vector
    [0.0, 0.0, 0.0],  # x-coordinates of repulsion vector
    [0.0, 0.0, 0.0],  # y-coordinates of repulsion vector
    [7.0, 7.0, 7.0],  # cohesion field radii
    [5.0, 5.0, 5.0],  # repulsion field radii
])
xv = np.subtract.outer(b[POS_X], b[POS_X])  # compute all pairs x-differences
yv = np.subtract.outer(b[POS_Y], b[POS_Y])  # compute all pairs y-differences

# compute all pairwise vector magnitudes
mag = np.hypot(xv, yv)              # all pairs magnitudes

# compute the cohesion neighbours
coh_n = mag <= b[CF]               # test for those pairs of agents for which one is within the cohesion field of the other  
np.fill_diagonal(coh_n, False)     # no agent is a cohesion neighbour of itself
nr_coh_n = np.sum(coh_n, axis = 0) # number of cohesion neighbours

# compute the x-differences and y-differences for cohesion vectors
xv_coh = np.where(coh_n, xv, 0.0)
yv_coh = np.where(coh_n, yv, 0.0)

# compute the resultant cohesion vectors 
b[COH_X] = xv_coh.sum(axis=0)                       # sum the x-differences 
b[COH_Y] = yv_coh.sum(axis=0)                       # sum the y-differences
b[COH_X:COH_Y+1] /= np.maximum(nr_coh_n, 1)         # divide by the number of cohesion neighbours

# compute the repulsion neighbours
rep_n = mag <= b[RF]
np.fill_diagonal(rep_n, False)     # no agent is a repulsion neighbour of itself
nr_rep_n = np.sum(rep_n, axis = 0) # number of repulsion neighbours

# compute the x-differences and y-differences for repulsion vectors
rscalar = mag / b[RF] - 1
xv_rep = np.where(rep_n, xv * rscalar, 0.0)
yv_rep = np.where(rep_n, yv * rscalar, 0.0)

# compute the resultant repulsion vectors 
b[REP_X] = xv_rep.sum(axis=0)                       # sum the x-differences 
b[REP_Y] = yv_rep.sum(axis=0)                       # sum the y-differences
b[REP_X:REP_Y+1] /= np.maximum(nr_rep_n, 1)         # divide by the number of repulsion neighbours

# plot the results
ph.plot_vector([], limit=11.0)                          # just draw the grid - no vectors
plot_field(b[CF, 0], 'b--')                             # draw the cohesion field of b[0]
plot_field(b[RF, 0], 'r--')                             # draw the repulsion field of b[0]
plt.plot(b[POS_X], b[POS_Y], 'ko', markersize=2)        # draw the agent points 
ph.plot_vector([b[COH_X:COH_Y+1].T[0]], color=ph.green, newfig=False)   # draw the resultant cohesion vector for b[0]
ph.plot_vector([b[REP_X:REP_Y+1].T[0]], color=ph.red, newfig=False)     # draw the resultant repulsion vectors for b[0]

Of course, agents $b_1$ and $b_2$ also experience similar effects due to the agents in their vicinity. This is shown below.

In [ ]:
# Define some useful array row constants for agent attributes
POS_X  = 0    # x-coordinates of agents' position 
POS_Y  = 1    # y-coordinates of agents' position
COH_X  = 2    # x-coordinates of cohesion vectors
COH_Y  = 3    # y-coordinates of cohesion vectors
REP_X  = 4    # x-coordinates of repulsion vectors
REP_Y  = 5    # y-coordinates of repulsion vectors
CF     = 6    # cohesion field radii
RF     = 7    # repulsion field radii

b = np.array([
    [0.0, 1.0, -2.0], # x-coordinates of agents' position
    [0.0, 0.5, 3.0],  # y-coordinates of agents' position
    [0.0, 0.0, 0.0],  # x-coordinates of cohesion vector
    [0.0, 0.0, 0.0],  # y-coordinates of cohesion vector
    [0.0, 0.0, 0.0],  # x-coordinates of repulsion vector
    [0.0, 0.0, 0.0],  # y-coordinates of repulsion vector
    [7.0, 7.0, 7.0],  # cohesion field radii
    [5.0, 5.0, 5.0],  # repulsion field radii
])
xv = np.subtract.outer(b[POS_X], b[POS_X])  # compute all pairs x-differences
yv = np.subtract.outer(b[POS_Y], b[POS_Y])  # compute all pairs y-differences

# compute all pairwise vector magnitudes
mag = np.hypot(xv, yv)              # all pairs magnitudes

# compute the cohesion neighbours
coh_n = mag <= b[CF]               # test for those pairs of agents for which one is within the cohesion field of the other  
np.fill_diagonal(coh_n, False)     # no agent is a cohesion neighbour of itself
nr_coh_n = np.sum(coh_n, axis = 0) # number of cohesion neighbours

# compute the x-differences and y-differences for cohesion vectors
xv_coh = np.where(coh_n, xv, 0.0)
yv_coh = np.where(coh_n, yv, 0.0)

# compute the resultant cohesion vectors 
b[COH_X] = xv_coh.sum(axis=0)                       # sum the x-differences 
b[COH_Y] = yv_coh.sum(axis=0)                       # sum the y-differences
b[COH_X:COH_Y+1] /= np.maximum(nr_coh_n, 1)         # divide by the number of cohesion neighbours

# compute the repulsion neighbours
rep_n = mag <= b[RF]
np.fill_diagonal(rep_n, False)     # no agent is a repulsion neighbour of itself
nr_rep_n = np.sum(rep_n, axis = 0) # number of repulsion neighbours

# compute the x-differences and y-differences for repulsion vectors
rscalar = mag / b[RF] - 1
xv_rep = np.where(rep_n, xv * rscalar, 0.0)
yv_rep = np.where(rep_n, yv * rscalar, 0.0)

# compute the resultant repulsion vectors 
b[REP_X] = xv_rep.sum(axis=0)                       # sum the x-differences 
b[REP_Y] = yv_rep.sum(axis=0)                       # sum the y-differences
b[REP_X:REP_Y+1] /= np.maximum(nr_rep_n, 1)         # divide by the number of repulsion neighbours

# plot the results
ph.plot_vector([], limit=11.0)                          # just draw the grid - no vectors
plot_field(b[CF, 0], 'b--')                             # draw the cohesion field of b[0]
plot_field(b[RF, 0], 'r--')                             # draw the repulsion field of b[0]
plt.plot(b[POS_X], b[POS_Y], 'ko', markersize=2)        # draw the agent points 
tails = b[POS_X:POS_Y+1].T                              # each agent acts as the 'tail' for its cohesion and repulsion vectors
coh_vectors = np.stack((xv_coh.T, yv_coh.T), axis=2)    # compute the targets of the cohesion vectors for each agent
for i in range(len(coh_vectors)):                       # plot the cohesion vectors for all agents
    ph.plot_vector(coh_vectors[i], [tails[i]], color=ph.green, newfig=False)   # draw the cohesion vectors
rep_vectors = np.stack((xv_rep.T, yv_rep.T), axis=2)    # compute the targets of the repulsion vectors for each agent
for i in range(len(rep_vectors)):                       # plot the repulsion vectors for all agents
    ph.plot_vector(rep_vectors[i], [tails[i]], color=ph.red, newfig=False)   # draw the repulsion vectors

## Resultant of cohesion and repulsion vectors

The movement of each agent is influenced by the sum of its cohesion and repulsion vectors. This sum is known as the *resultant* of the cohesion and repulsion vectors. Here we add 2 additional rows to the swarm state array to store the x- and y-coordinates of the overall resultant vectors.

In [ ]:
# Define some useful array row constants for agent attributes
POS_X  = 0    # x-coordinates of agents' position 
POS_Y  = 1    # y-coordinates of agents' position
COH_X  = 2    # x-coordinates of cohesion vectors
COH_Y  = 3    # y-coordinates of cohesion vectors
REP_X  = 4    # x-coordinates of repulsion vectors
REP_Y  = 5    # y-coordinates of repulsion vectors
RES_X  = 6    # x-coordinates of resultant vectors
RES_Y  = 7    # y-coordinates of resultant vectors
CF     = 8    # cohesion field radii
RF     = 9    # repulsion field radii

b = np.array([
    [0.0, 1.0, -2.0], # x-coordinates of agents' position
    [0.0, 0.5, 3.0],  # y-coordinates of agents' position
    [0.0, 0.0, 0.0],  # x-coordinates of cohesion vector
    [0.0, 0.0, 0.0],  # y-coordinates of cohesion vector
    [0.0, 0.0, 0.0],  # x-coordinates of repulsion vector
    [0.0, 0.0, 0.0],  # y-coordinates of repulsion vector
    [0.0, 0.0, 0.0],  # x-coordinates of resultant vector
    [0.0, 0.0, 0.0],  # y-coordinates of resultant vector
    [7.0, 7.0, 7.0],  # cohesion field radii
    [5.0, 5.0, 5.0],  # repulsion field radii
])
xv = np.subtract.outer(b[POS_X], b[POS_X])  # compute all pairs x-differences
yv = np.subtract.outer(b[POS_Y], b[POS_Y])  # compute all pairs y-differences

# compute all pairwise vector magnitudes
mag = np.hypot(xv, yv)              # all pairs magnitudes

# compute the cohesion neighbours
coh_n = mag <= b[CF]               # test for those pairs of agents for which one is within the cohesion field of the other  
np.fill_diagonal(coh_n, False)     # no agent is a cohesion neighbour of itself
nr_coh_n = np.sum(coh_n, axis = 0) # number of cohesion neighbours

# compute the x-differences and y-differences for cohesion vectors
xv_coh = np.where(coh_n, xv, 0.0)
yv_coh = np.where(coh_n, yv, 0.0)

# compute the resultant cohesion vectors 
b[COH_X] = xv_coh.sum(axis=0)                       # sum the x-differences 
b[COH_Y] = yv_coh.sum(axis=0)                       # sum the y-differences
b[COH_X:COH_Y+1] /= np.maximum(nr_coh_n, 1)         # divide by the number of cohesion neighbours

# compute the repulsion neighbours
rep_n = mag <= b[RF]
np.fill_diagonal(rep_n, False)     # no agent is a repulsion neighbour of itself
nr_rep_n = np.sum(rep_n, axis = 0) # number of repulsion neighbours

# compute the x-differences and y-differences for repulsion vectors
rscalar = mag / b[RF] - 1
xv_rep = np.where(rep_n, xv * rscalar, 0.0)
yv_rep = np.where(rep_n, yv * rscalar, 0.0)

# compute the resultant repulsion vectors 
b[REP_X] = xv_rep.sum(axis=0)                       # sum the x-differences 
b[REP_Y] = yv_rep.sum(axis=0)                       # sum the y-differences
b[REP_X:REP_Y+1] /= np.maximum(nr_rep_n, 1)         # divide by the number of repulsion neighbours

# compute the overall resultant of the cohesion and repulsion vectors
b[RES_X:RES_Y+1] = b[COH_X:COH_Y+1] + b[REP_X:REP_Y+1] 

# plot the results
ph.plot_vector([], limit=11.0)                          # just draw the grid - no vectors
plot_field(b[CF, 0], 'b--')                             # draw the cohesion field of b[0]
plot_field(b[RF, 0], 'r--')                             # draw the repulsion field of b[0]
plt.plot(b[POS_X], b[POS_Y], 'ko', markersize=2)        # draw the agent points 
ph.plot_vector(b[RES_X:RES_Y+1].T, b[POS_X:POS_Y+1].T, newfig=False)   # draw the resultant vectors for all agents

## Random swarms, goals, direction vectors and weights

So far, we have considered only swarms in which all agents are within the cohesion and repulsion fields of each other. It is very unlikely that this will be the case in general. We introduce a function `mk_rand_swarm()` to create swarms in which the initial placement of agents inside or outside the cohesion or repulsion fields of other agents occurs randomly. 

Two further ideas are introduced:

    1. a goal position for the swarm to settle on, inducing a direction vector for each agent; 
    2. weighting factors to allow different weights to be placed on the cohesion, repulsion and direction components of the overall
       resultant vector for each agent.
       
Notice that a goal, $g$, induces a direction vector, $\overrightarrow{b g}$, for each agent $b$. As usual, the vector $\overrightarrow{b g}$ is computed as $g - b$. Now, the overall resultant vector for each agent, $b$, is computed as the weighted sum of its cohesion, repulsion and direction vectors.

In [ ]:
# Define some useful array accessor constants
POS_X  = 0    # x-coordinates of agents position 
POS_Y  = 1    # y-coordinates of agents position
COH_X  = 2    # x-coordinates of cohesion vectors
COH_Y  = 3    # y-coordinates of cohesion vectors
REP_X  = 4    # x-coordinates of repulsion vectors
REP_Y  = 5    # y-coordinates of repulsion vectors
DIR_X  = 6    # x-coordinates of direction vectors
DIR_Y  = 7    # y-coordinates of direction vectors
RES_X  = 8    # x-coordinates of resultant vectors
RES_Y  = 9    # y-coordinates of resultant vectors
GOAL_X = 10   # x-coordinates of goals
GOAL_Y = 11   # y-coordinates of goals
CF     = 12   # cohesion field radii
RF     = 13   # repulsion field radii
KC     = 14   # cohesion vector scaling factor
KR     = 15   # repulsion vector scaling factor
KD     = 16   # direction vector scaling factor

N_ROWS = 17   # number of rows in array that models swarm state

def mk_rand_swarm(n, *, cf=4.0, rf=3.0, kc=1.0, kr=1.0, kd=0.0, goal=0.0, loc=0.0, grid=10):
    '''
    create a 2-D array of N_ROWS attributes for n agents. 
    
    :param n:      number of agents
    :param cf:     cohesion field radius of all agents; default 4.0; heterogeneous fields are allowed but not catered for here
    :param rf:     repulsion field radius of all agents; default 3.0
    :param kc:     weighting factor for cohesion component, default 1.0
    :param kr:     weighting factor for repulsion component, default 1.0
    :param kd:     weighting factor for direction component, default 0.0 (i.e. goal is ignored by default)
    :param goal:   location of a goal for all agents; heterogeneous goals are allowed but not catered for here
    :param loc:    location of agent b_0
    :param grid:   size of grid around b_0 in which all other agents will be placed initially at random
    '''
    b = np.empty((N_ROWS, n))                       #create a 2-D array, big enough for n agents
    b[POS_X:POS_Y + 1,:] = (np.random.uniform(size=2 * n) * 2 * grid - grid + loc).reshape(2, n) # place agents randomly
    b[POS_X:POS_Y + 1,0] = loc                      # b_0 placed at [loc, loc]       
    b[COH_X:COH_Y+1,:] = np.full((2,n), 0.0)        # cohesion vectors initially [0.0, 0.0]
    b[REP_X:REP_Y+1,:] = np.full((2,n), 0.0)        # repulsion vectors initially [0.0, 0.0]
    b[DIR_X:DIR_Y+1,:] = np.full((2,n), 0.0)        # direction vectors initially [0.0, 0.0]
    b[RES_X:RES_Y + 1,:] = np.full((2,n), 0.0)      # resultant vectors initially [0.0, 0.0]
    b[GOAL_X:GOAL_Y + 1,:] = np.full((2,n), goal)   # goal is at [goal, goal], default [0.0, 0.0]
    b[CF,:] = np.full(n, cf)                        # cohesion field of all agents set to cf
    b[RF,:] = np.full(n, rf)                        # repulsion field of all agents set to rf
    b[KC,:] = np.full(n, kc)                        # cohesion weight for all agents set to kc
    b[KR,:] = np.full(n, kr)                        # repulsion weight for all agents set to kr
    b[KD,:] = np.full(n, kd)                        # direction weight for all agents set to kd
    return b

Let's try creating a small random swarm and plotting the resultant vectors arising from a single step of all agents. You can play around with the parameters for `mk_rand_swarm()` here and observe the effects. Try varying the size of the fields, the weightings, the starting location and the goal.

In [ ]:
# create a small, simple, random swarm
b = mk_rand_swarm(10, grid=4.0)

# set `loc` to locate the initial swarm somewhere other than the origin
# set `goal` to give the swarm a goal
# set `kf` to a value >0 to include the direction vectors in the resultant
# set `grid` to reduce the size of the initial grid
# b = mk_rand_swarm(10, loc=-7.5, goal=-5.0, kd=1.0, grid=2.0)

# Compute the resultant vectors, including direction vectors and weightings

xv = np.subtract.outer(b[POS_X], b[POS_X])  # compute all pairs x-differences
yv = np.subtract.outer(b[POS_Y], b[POS_Y])  # compute all pairs y-differences

# compute all pairwise vector magnitudes
mag = np.hypot(xv, yv)              # all pairs magnitudes

# compute the cohesion neighbours
coh_n = mag <= b[CF]               # test for those pairs of agents for which one is within the cohesion field of the other  
np.fill_diagonal(coh_n, False)     # no agent is a cohesion neighbour of itself
nr_coh_n = np.sum(coh_n, axis = 0) # number of cohesion neighbours

# compute the x-differences and y-differences for cohesion vectors
xv_coh = np.where(coh_n, xv, 0.0)
yv_coh = np.where(coh_n, yv, 0.0)

# compute the resultant cohesion vectors 
b[COH_X] = xv_coh.sum(axis=0)                       # sum the x-differences 
b[COH_Y] = yv_coh.sum(axis=0)                       # sum the y-differences
b[COH_X:COH_Y+1] /= np.maximum(nr_coh_n, 1)         # divide by the number of cohesion neighbours

# compute the repulsion neighbours
rep_n = mag <= b[RF]
np.fill_diagonal(rep_n, False)     # no agent is a repulsion neighbour of itself
nr_rep_n = np.sum(rep_n, axis = 0) # number of repulsion neighbours

# compute the x-differences and y-differences for repulsion vectors
rscalar = mag / b[RF] - 1     # repulsion scaling factor
xv_rep = np.where(rep_n, xv * rscalar, 0.0)
yv_rep = np.where(rep_n, yv * rscalar, 0.0)

# compute the resultant repulsion vectors 
b[REP_X] = xv_rep.sum(axis=0)                       # sum the x-differences 
b[REP_Y] = yv_rep.sum(axis=0)                       # sum the y-differences
b[REP_X:REP_Y+1] /= np.maximum(nr_rep_n, 1)         # divide by the number of repulsion neighbours

# compute the direction vectors
b[DIR_X:DIR_Y+1] = b[GOAL_X:GOAL_Y+1] - b[POS_X:POS_Y+1]

# compute the resultant of the weighted cohesion, repulsion and direction vectors
b[RES_X:RES_Y+1] = b[KC] * b[COH_X:COH_Y+1] + b[KR] * b[REP_X:REP_Y+1] + b[KD] * b[DIR_X:DIR_Y+1]

# plot the results
ph.plot_vector([], limit=11.0)                            # just draw the grid - no vectors
plot_field(b[CF, 0], 'b--', pos=(b[POS_X,0], b[POS_Y,0])) # draw the cohesion field of b[0]
plot_field(b[RF, 0], 'r--', pos=(b[POS_X,0], b[POS_Y,0])) # draw the repulsion field of b[0]
plt.plot(b[POS_X], b[POS_Y], 'ko', markersize=2)          # draw the agent points 
ph.plot_vector(b[RES_X:RES_Y+1].T, b[POS_X:POS_Y+1].T, newfig=False)   # draw the resultant vectors for all agents

## A basic simulator

A simple swarm simulator can be developed by defining a `step()` function, along the lines that we have followed so far. The only additional requirement is to normalise the overall resultant vector for each agent and to apply a speed factor to the normalised vector to calculate the distance moved by each agent in a single unit of simulated time (a step). 

Note the normalised vector derived from $\overrightarrow{v}$ is $\hat{v}$, where $\hat{v} = \frac{\overrightarrow{v}}{\lVert\overrightarrow{v}\rVert}$, giving a vector that has the same direction as $\overrightarrow{v}$ but with magnitude $1.0$.

In [ ]:
def step(b, *, speed=0.05):
    """
    Compute one step in the evolution of swarm `b`
    
    :param b: the array modelling the state of the swarm
    :param speed: the speed of each agent, i.e. the number of simulation distance units per simulation time unit (step), default 0.05.
    """

    xv = np.subtract.outer(b[POS_X], b[POS_X])  # compute all pairs x-differences
    yv = np.subtract.outer(b[POS_Y], b[POS_Y])  # compute all pairs y-differences

    # compute all pairwise vector magnitudes
    mag = np.hypot(xv, yv)              # all pairs magnitudes

    # compute the cohesion neighbours
    coh_n = mag <= b[CF]               # test for those pairs of agents for which one is within the cohesion field of the other  
    np.fill_diagonal(coh_n, False)     # no agent is a cohesion neighbour of itself
    nr_coh_n = np.sum(coh_n, axis = 0) # number of cohesion neighbours

    # compute the x-differences and y-differences for cohesion vectors
    xv_coh = np.where(coh_n, xv, 0.0)
    yv_coh = np.where(coh_n, yv, 0.0)

    # compute the resultant cohesion vectors 
    b[COH_X] = xv_coh.sum(axis=0)                       # sum the x-differences 
    b[COH_Y] = yv_coh.sum(axis=0)                       # sum the y-differences
    b[COH_X:COH_Y+1] /= np.maximum(nr_coh_n, 1)         # divide by the number of cohesion neighbours

    # compute the repulsion neighbours
    rep_n = mag <= b[RF]
    np.fill_diagonal(rep_n, False)     # no agent is a repulsion neighbour of itself
    nr_rep_n = np.sum(rep_n, axis = 0) # number of repulsion neighbours

    # compute the x-differences and y-differences for repulsion vectors
    rscalar = mag / b[RF] - 1                           # repulsion scaling factor
    xv_rep = np.where(rep_n, xv * rscalar, 0.0)
    yv_rep = np.where(rep_n, yv * rscalar, 0.0)
    
    # compute the resultant repulsion vectors 
    b[REP_X] = xv_rep.sum(axis=0)                       # sum the x-differences 
    b[REP_Y] = yv_rep.sum(axis=0)                       # sum the y-differences
    b[REP_X:REP_Y+1] /= np.maximum(nr_rep_n, 1)         # divide by the number of repulsion neighbours

    # compute the direction vectors
    b[DIR_X:DIR_Y+1] = b[GOAL_X:GOAL_Y+1] - b[POS_X:POS_Y+1]

    # compute the resultant of the weighted cohesion, repulsion and direction vectors
    b[RES_X:RES_Y+1] = b[KC] * b[COH_X:COH_Y+1] + b[KR] * b[REP_X:REP_Y+1] + b[KD] * b[DIR_X:DIR_Y+1]
        
    # normalise the resultant
    mag_res = np.hypot(b[RES_X], b[RES_Y])
    b[RES_X:RES_Y+1, mag_res == 0] = 0.
    b[RES_X:RES_Y+1, mag_res != 0] /= mag_res[mag_res != 0]

    # multiply resultant by factor for speed and update positions of agents
    b[RES_X:RES_Y+1] *= speed                            # scale normalised resultant vector by speed
    b[POS_X:POS_Y+1] += b[RES_X:RES_Y+1]                 # update positions of all agents

Given a `step()` function, we can create a function, `run_simulation()`, to run a simulation of the `step()` function. The `run_simulation()` function is very simple. It takes a swarm as its first argument, creates a simple figure to record the positions of the swarm's agents, repeatedly runs the `step()` function, and updates the figure with the agents recorded at their new positions. The simplicity of the `run_simulation()` function derives from its use of `matplotlib.animation.FuncAnimation` which handles the periodic running of the `step()` function and the updating of the figure.

In [ ]:
def run_simulation(b, *, step=step):
    """
    run a simulation of the `step()` function in a simple graphical environment
    """
    fig, ax = plt.subplots(figsize=(4,4))                  # create a graph
    ax.set(xlim=(-10, 10), ylim=(-10, 10))                 # set the limits of the axes
    agent_data, = ax.plot(b[0], b[1], 'ko', markersize=2)  # plot the initial locations of the agents

    def simulate(i):
        """
        Ultra-simple simulation function  
        """
        step(b)                                         # take a step
        agent_data.set_data(b[0], b[1])                 # plot the agents at their new locations
        
    return FuncAnimation(fig, simulate, interval=100)

Now we can create a variety of small- to medium-sized swarms and observe their behaviour. Importantly, however, the use of Numpy's vectorised operations provides the basis of an efficient approach to the simulation of much larger swarms. This will be explored in more detail later. For now, you should create some simple swarms, using a variety of parameters, to see how they behave.

In [ ]:
# create a swarm using some interesting parameters
# b = mk_rand_swarm(100, rf=4.0, cf=5.0, grid=4.0)
# b = mk_rand_swarm(100, kd=1.0, kr=4.0, loc=-7.0, grid=4.0)
# b = mk_rand_swarm(100, rf=1.0, cf=2.0, kr=15.0, grid=4.0)
# b = mk_rand_swarm(100, rf=3.0, cf=5.0, grid=10.0)
# b = mk_rand_swarm(100, rf=2.0, cf=5.0, kr=30.0, grid=6.0)
b = mk_rand_swarm(36, loc=-7.5, kd=1.0, kr=30.0, grid=4.0)
sim = run_simulation(b, step=step)

We can make a preliminary stab at estimating the performance of our `step` algorithm by creating a random swarm of 1000 agents and using the `%timeit` magic function to see how long it takes to execute a step in the evolution of the swarm.

In [ ]:
b = mk_rand_swarm(1000)
%timeit step(b)

We can obtain a more detailed understanding of the performance of the code by using the `line_profiler`. You can install this for `anaconda` using `conda install line_profiler`.

In [ ]:
%load_ext line_profiler

In [ ]:
b = mk_rand_swarm(2000)    # create a swarm of 2000 agents
%lprun -f step step(b)     # get a line-by-line breakdown of where the time is used

## A revised repulsion scaling factor

### Background

The `step()` function is intended to implement the mathematical specification of swarm behaviour given in (Eliot, 2017). To paraphrase briefly this specification (ignoring obstacles for the moment) for a swarm, $S$, and $b \in S$:

$$
v(b) = k_cv_c(b) + k_rv_r(b) + k_dv_d(b)
$$
where $k_c$, $k_r$, and $k_d$ are the weighting factors for the cohesion, repulsion and direction components of the resultant vector, respectively, and:

$$
v_c(b) = \frac{1}{\lvert\mathit{coh}(b)\rvert} \sum_{b' \in \mathit{coh}(b)}\overrightarrow{b b'}
$$

$$
v_r(b) = \frac{-1}{\lvert\mathit{rep}(b)\rvert} \sum_{b' \in \mathit{rep}(b)}\left(\left(1 - \frac{\lVert\overrightarrow{b b'}\rVert}{R_b}\right)\overrightarrow{b b'}\right)
$$

$$
v_d(b) = \overrightarrow{b g}
$$

where $\mathit{coh}(b)$ and $\mathit{rep}(b)$ are functions returning the sets of cohesion and repulsion neighbours of $b$, as follows:

$$
\mathit{coh}(b) = \{b' \in S : b' \neq b \land \lVert\overrightarrow{b b'}\rVert <= C_b\}
$$

$$
\mathit{rep}(b) = \{b' \in S : b' \neq b \land \lVert\overrightarrow{b b'}\rVert <= R_b\}
$$

and $g$ specifies the Cartesian coordinates of a position in the 2-D space of the swarm, known as the 'goal'. The implementation of `step()` follows this specification directly in the cases of of $v_c(b)$ and $v_d(b)$. The `step()` function implements a rewritten, but equivalent, specification of $v_r(b)$ , namely:

$$
v_r(b) = \frac{1}{\lvert\mathit{rep}(b)\rvert} \sum_{b' \in \mathit{rep}(b)}\left(\left(\frac{\lVert\overrightarrow{b b'}\rVert}{R_b} - 1\right)\overrightarrow{b b'}\right)
$$

### The problem

It is easy to explore the import of this specification for a pair of distinct agents, $b$ and $b'$, simply by focusing on the expression

$$
\left(\left(\frac{\lVert\overrightarrow{b b'}\rVert}{R_b} - 1\right)\overrightarrow{b b'}\right)
$$

Consider an agent $b_0$ at the origin with a repulsion field $R_{b_0}$ of $5.0$ and a second agent $b_1$ at $(4,3)$. The magnitude of the vector $\overrightarrow{b_0 b_1}$ is 5.0, so the magnitude of the resulting repulsion vector is $0$, as shown below:

In [ ]:
# Define some useful array row constants for agent attributes
POS_X  = 0    # x-coordinates of agents' position 
POS_Y  = 1    # y-coordinates of agents' position
COH_X  = 2    # x-coordinates of cohesion vectors
COH_Y  = 3    # y-coordinates of cohesion vectors
REP_X  = 4    # x-coordinates of repulsion vectors
REP_Y  = 5    # y-coordinates of repulsion vectors
CF     = 6    # cohesion field radii
RF     = 7    # repulsion field radii

b = np.array([
    [0.0, 4.0], # x-coordinates of agents' position
    [0.0, 3.0], # y-coordinates of agents' position
    [0.0, 0.0], # x-coordinates of cohesion vector
    [0.0, 0.0], # y-coordinates of cohesion vector
    [0.0, 0.0], # x-coordinates of repulsion vector
    [0.0, 0.0], # y-coordinates of repulsion vector
    [7.0, 7.0], # cohesion field radii
    [5.0, 5.0], # repulsion field radii
])
xv = np.subtract.outer(b[POS_X], b[POS_X])  # compute all pairs x-differences
yv = np.subtract.outer(b[POS_Y], b[POS_Y])  # compute all pairs y-differences

# compute all pairwise vector magnitudes
mag = np.hypot(xv, yv)              # all pairs magnitudes

# compute the repulsion neighbours
rep_n = mag <= b[RF]
np.fill_diagonal(rep_n, False)     # no agent is a repulsion neighbour of itself
nr_rep_n = np.sum(rep_n, axis = 0) # number of repulsion neighbours

# compute the x-differences  and y-differences for repulsion vectors
rscalar = mag / b[RF] - 1
xv_rep = np.where(rep_n, xv * rscalar, 0.0)
yv_rep = np.where(rep_n, yv * rscalar, 0.0)

# plot the results
ph.plot_vector([], limit=11.0)                                        # just draw the grid - no vectors
plot_field(b[CF, 0], 'b--')                                           # draw the cohesion field of b[0]
plot_field(b[RF, 0], 'r--')                                           # draw the repulsion field of b[0]
plt.plot(b[POS_X], b[POS_Y], 'ko', markersize=2)                      # draw the agent points 
rep_v_0 = np.column_stack((xv_rep.T[0], yv_rep.T[0]))                 # get repulsion vectors for agent 0 - just for plotting            
ph.plot_vector(rep_v_0, color=ph.red, newfig=False)                   # draw the repulsion vector from b[0] to b[1]
plt.text(0.1, -0.6, 'b0')
plt.text(b[POS_X,1] - 0.8, b[POS_Y,1] - 0.2, 'b1')
print(f"The magnitude of the vector b0b1 is {mag[1, 0]}")
print(f"The magnitude of the repulsion vector is {np.hypot(xv_rep[1,0], yv_rep[1,0])}")

Now try changing the location of $b_1$ to (3, 2.25) in the code above. Run it and observe the effect. We now have a repulsion vector whose magnitude is 0.9375. Now move $b_1$ to (2, 1.5). Run and observe as before. The magnitude of the repulsion vector is now 1.25. But, from this point on, as $b_1$ moves closer to $b_0$, the magnitude of the repulsion vector grows *smaller*. Try $b_1$ at (1, 0.75). The magnitude of the repulsion vector is now back down to 0.9375. What about (0.5, 0.375)? Tha magnitude of the repulsion vector is 0.546875! In fact, the magnitude of the repulsion vector is at its maximum when the distance between $b$ and $b'$ is $\frac{R_b}{2}$ and *decreases* then, as the distance between the agents decreases. Surely, this is not how we expect the repulsion vector to behave?

### A proposed solution

An approach in which the magnitude of the repulsion vector continues to grow as an agent moves closer would seem to be preferable. There is a variety of possible ways to achieve this. One simple one is introduced here. The idea is to use the radius of the repulsion field to scale the normalised vector between each agent pair, as follows:

$$
v_r(b) = \frac{1}{\lvert\mathit{rep}(b)\rvert} \sum_{b' \in \mathit{rep}(b)}\left(\left(\lVert\overrightarrow{b b'}\rVert - R_b\right)\widehat{b b'}\right)
$$

Remember that we only consider repulsion neighbours here. So, $\lVert\overrightarrow{b b'}\rVert \leq R_b$ and, therefore, $\lVert\overrightarrow{b b'}\rVert - R_b$ is either 0 or negative, and it becomes increasingly negative as the distance between the agents becomes smaller.


In [ ]:
# Define some useful array row constants for agent attributes
POS_X  = 0    # x-coordinates of agents' position 
POS_Y  = 1    # y-coordinates of agents' position
COH_X  = 2    # x-coordinates of cohesion vectors
COH_Y  = 3    # y-coordinates of cohesion vectors
REP_X  = 4    # x-coordinates of repulsion vectors
REP_Y  = 5    # y-coordinates of repulsion vectors
CF     = 6    # cohesion field radii
RF     = 7    # repulsion field radii

b = np.array([
    [0.0, 4.0], # x-coordinates of agents' position
    [0.0, 3.0], # y-coordinates of agents' position
    [0.0, 0.0], # x-coordinates of cohesion vector
    [0.0, 0.0], # y-coordinates of cohesion vector
    [0.0, 0.0], # x-coordinates of repulsion vector
    [0.0, 0.0], # y-coordinates of repulsion vector
    [7.0, 7.0], # cohesion field radii
    [5.0, 5.0], # repulsion field radii
])
xv = np.subtract.outer(b[POS_X], b[POS_X])  # compute all pairs x-differences
yv = np.subtract.outer(b[POS_Y], b[POS_Y])  # compute all pairs y-differences

# compute all pairwise vector magnitudes
mag = np.hypot(xv, yv)              # all pairs magnitudes

# compute the repulsion neighbours
rep_n = mag <= b[RF]
np.fill_diagonal(rep_n, False)     # no agent is a repulsion neighbour of itself
nr_rep_n = np.sum(rep_n, axis = 0) # number of repulsion neighbours

# compute the x-differences and y-differences for repulsion vectors
mag_nz = mag != 0                                     
rep_n_and_mag_nz = np.logical_and(rep_n, mag_nz)    # repulsion neighbour and magnitude non-zero
rscalar = mag - b[RF]                             # repulsion scaling factor
xv_rep = np.where(rep_n_and_mag_nz, np.divide(xv * rscalar, mag, where = mag_nz), 0.) # ensure 0 if not repulsion neighbour or mag = 0
yv_rep = np.where(rep_n_and_mag_nz, np.divide(yv * rscalar, mag, where = mag_nz), 0.) # ensure 0 if not repulsion neighbour or mag = 0

# plot the results
ph.plot_vector([], limit=11.0)                                        # just draw the grid - no vectors
plot_field(b[CF, 0], 'b--')                                           # draw the cohesion field of b[0]
plot_field(b[RF, 0], 'r--')                                           # draw the repulsion field of b[0]
plt.plot(b[POS_X], b[POS_Y], 'ko', markersize=2)                      # draw the agent points 
rep_v_0 = np.column_stack((xv_rep.T[0], yv_rep.T[0]))                 # get repulsion vectors for agent 0 - just for plotting            
ph.plot_vector(rep_v_0, color=ph.red, newfig=False)                   # draw the repulsion vector from b[0] to b[1]
plt.text(0.1, -0.6, 'b0')
plt.text(b[POS_X,1] - 0.8, b[POS_Y,1] - 0.2, 'b1')
print(f"The magnitude of the vector b0b1 is {mag[1, 0]}")
print(f"The magnitude of the repulsion vector is {np.hypot(xv_rep[1,0], yv_rep[1,0])}")

As before, try changing the location of $b_1$ to (3, 2.25), (2, 1.5), (1, 0.75), and (0.5, 0.375), and, in each case, observe the magnitude of the repulsion vector induced in $b_0$. This looks like a more sensible approach. It may be worth experimenting with vaiations on this, for example, using constant, quadratic or cubic factors, but, so far, I haven't found anything that improves behaviour significantly beyond this simple linear approach.

The program below illustrates the kind of resultant vectors generated using this new approach.

In [ ]:
# create a small, simple, random swarm
b = mk_rand_swarm(10, grid=4.0)

# set `loc` to locate the initial swarm somewhere other than the origin
# set `goal` to give the swarm a goal
# set `kf` to a value >0 to include the direction vectors in the resultant
# set `grid` to reduce the size of the initial grid
# b = mk_rand_swarm(10, loc=-7.5, goal=-5.0, kd=1.0, grid=2.0)

# Compute the resultant vectors, including direction vectors and weightings

xv = np.subtract.outer(b[POS_X], b[POS_X])  # compute all pairs x-differences
yv = np.subtract.outer(b[POS_Y], b[POS_Y])  # compute all pairs y-differences

# compute all pairwise vector magnitudes
mag = np.hypot(xv, yv)              # all pairs magnitudes

# compute the cohesion neighbours
coh_n = mag <= b[CF]               # test for those pairs of agents for which one is within the cohesion field of the other  
np.fill_diagonal(coh_n, False)     # no agent is a cohesion neighbour of itself
nr_coh_n = np.sum(coh_n, axis = 0) # number of cohesion neighbours

# compute the x-differences and y-differences for cohesion vectors
xv_coh = np.where(coh_n, xv, 0.0)
yv_coh = np.where(coh_n, yv, 0.0)

# compute the resultant cohesion vectors 
b[COH_X] = xv_coh.sum(axis=0)                       # sum the x-differences 
b[COH_Y] = yv_coh.sum(axis=0)                       # sum the y-differences
b[COH_X:COH_Y+1] /= np.maximum(nr_coh_n, 1)                           # divide by the number of cohesion neighbours

# compute the repulsion neighbours
rep_n = mag <= b[RF]
np.fill_diagonal(rep_n, False)     # no agent is a repulsion neighbour of itself
nr_rep_n = np.sum(rep_n, axis = 0) # number of repulsion neighbours

# compute the x-differences and y-differences for repulsion vectors
mag_nz = mag != 0                                     
needed_and_safe = np.logical_and(rep_n, mag_nz)  # repulsion neighbour and magnitude non-zero
rscalar = mag - b[RF]                            # repulsion scaling factor
xv_rep = np.where(needed_and_safe, np.divide(xv * rscalar, mag, where = mag_nz), 0.) # ensure 0 if not repulsion neighbour or mag = 0
yv_rep = np.where(needed_and_safe, np.divide(yv * rscalar, mag, where = mag_nz), 0.) # ensure 0 if not repulsion neighbour or mag = 0

# compute the resultant repulsion vectors 
b[REP_X] = xv_rep.sum(axis=0)                       # sum the x-differences 
b[REP_Y] = yv_rep.sum(axis=0)                       # sum the y-differences
b[REP_X:REP_Y+1] /= np.maximum(nr_rep_n, 1)            # divide by the number of repulsion neighbours

# compute the direction vectors
b[DIR_X:DIR_Y+1] = b[GOAL_X:GOAL_Y+1] - b[POS_X:POS_Y+1]

# compute the resultant of the weighted cohesion, repulsion and direction vectors
b[RES_X:RES_Y+1] = b[KC] * b[COH_X:COH_Y+1] + b[KR] * b[REP_X:REP_Y+1] + b[KD] * b[DIR_X:DIR_Y+1]

# plot the results
ph.plot_vector([], limit=11.0)                            # just draw the grid - no vectors
plot_field(b[CF, 0], 'b--', pos=(b[POS_X,0], b[POS_Y,0])) # draw the cohesion field of b[0]
plot_field(b[RF, 0], 'r--', pos=(b[POS_X,0], b[POS_Y,0])) # draw the repulsion field of b[0]
plt.plot(b[POS_X], b[POS_Y], 'ko', markersize=2)          # draw the agent points 
ph.plot_vector(b[RES_X:RES_Y+1].T, b[POS_X:POS_Y+1].T, newfig=False)   # draw the resultant vectors for all agents

The code below implements a step function, `d_step()`, that implements the new approach. Try plugging this into some simulations.

In [ ]:
def d_step(b, *, speed=0.05):
    xv = np.subtract.outer(b[POS_X], b[POS_X])  # all pairs x-differences
    yv = np.subtract.outer(b[POS_Y], b[POS_Y])  # all pairs y-differences

    # compute all pairwise vector magnitudes
    mag = np.hypot(xv, yv)              # all pairs magnitudes
    
    # compute the cohesion neighbours
    coh_n = mag <= b[CF]
    np.fill_diagonal(coh_n, False)     # no agent is a cohesion neighbour of itself
    nr_coh_n = np.sum(coh_n, axis = 0) # number of cohesion neighbours

    # compute the x-differences and y-differences for cohesion vectors
    xv_coh = np.where(coh_n, xv, 0.0)
    yv_coh = np.where(coh_n, yv, 0.0)

    # compute the cohesion vectors 
    b[COH_X] = xv_coh.sum(axis=0)                       # sum the x-differences 
    b[COH_Y] = yv_coh.sum(axis=0)                       # sum the y-differences
    b[COH_X:COH_Y+1] /= np.maximum(nr_coh_n, 1)         # divide by the number of cohesion neighbours

    # compute the repulsion neighbours
    rep_n = mag <= b[RF]
    np.fill_diagonal(rep_n, False)     # no agent is a repulsion neighbour of itself
    nr_rep_n = np.sum(rep_n, axis = 0) # number of repulsion neighbours

    # compute the x-differences and y-differences for repulsion vectors
    mag_nz = mag != 0                                     
    rep_n_and_mag_nz = np.logical_and(rep_n, mag_nz)    # repulsion neighbour and magnitude non-zero
    rscalar = mag - b[RF]                             # repulsion scaling factor
    xv_rep = np.where(rep_n_and_mag_nz, np.divide(xv * rscalar, mag, where = mag_nz), 0.) # ensure 0 if not repulsion neighbour or mag = 0
    yv_rep = np.where(rep_n_and_mag_nz, np.divide(yv * rscalar, mag, where = mag_nz), 0.) # ensure 0 if not repulsion neighbour or mag = 0

    # compute the resultant repulsion vectors 
    b[REP_X] = xv_rep.sum(axis=0)                       # sum the x-differences 
    b[REP_Y] = yv_rep.sum(axis=0)                       # sum the y-differences
    b[REP_X:REP_Y+1] /= np.maximum(nr_rep_n, 1)         # divide by the number of repulsion neighbours
    
    # compute the direction vectors
    b[DIR_X:DIR_Y+1] = b[GOAL_X:GOAL_Y+1] - b[POS_X:POS_Y+1]

    # compute the resultant of the cohesion, repulsion and direction vectors
    b[RES_X:RES_Y+1] = b[KC] * b[COH_X:COH_Y+1] + b[KR] * b[REP_X:REP_Y+1] + b[KD] * b[DIR_X:DIR_Y+1]
                  
    # compute the resultant magnitudes and normalise the resultant
    mag_res = np.hypot(b[RES_X], b[RES_Y])
    b[RES_X:RES_Y+1, mag_res == 0] = 0
    b[RES_X:RES_Y+1, mag_res != 0] /= mag_res[mag_res != 0]

    # multiply resultant by factor for speed and update positions of agents
    b[RES_X:RES_Y+1] *= speed                               # distance units per time unit
    b[POS_X:POS_Y+1] += b[RES_X:RES_Y+1]                    # update positions


In [ ]:
# create a swarm using some interesting parameters
# b = mk_rand_swarm(100, rf=4.0, cf=5.0, grid=4.0)
# b = mk_rand_swarm(100, kd=1.0, kr=4.0, loc=-7.0, grid=4.0)
# b = mk_rand_swarm(100, rf=1.0, cf=2.0, kr=15.0, grid=4.0)
# b = mk_rand_swarm(100, rf=3.0, cf=5.0, grid=10.0)
# b = mk_rand_swarm(100, rf=2.0, cf=5.0, kr=30.0, grid=6.0)
b = mk_rand_swarm(36, loc=-7.5, kd=1.0, kr=30.0, grid=4.0)
sim = run_simulation(b, step=d_step)

The new `d_step()` function is about 10% slower than the previous `step()` function

In [ ]:
b = mk_rand_swarm(1000)
%timeit d_step(b)

In [ ]:
b = mk_rand_swarm(2000)    # create a swarm of 2000 agents
%lprun -f d_step d_step(b)     # get a line-by-line breakdown of where the time is used